# Rossmann Pharmaceuticals

#### This script is to conduct prediction of 2 weeks sales across stores of Rossmann Pharmaceuticals.

### Outline

1. Task 2.1 - Preprocessing


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
warnings.filterwarnings('ignore')

# Add the parent directory to the Python path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
from scripts.load_data import load_data
logging.info("Loading data from csv file")
df_store= load_data('../data/store.csv')
df_test=load_data('../data/test.csv')
df_train=load_data('../data/train.csv')
logging.info("Data loaded successfully")

# Merging train data with store data to analyze store-level details
merged_data = pd.merge(df_train, df_store, on='Store')


2025-01-14 09:39:02,727 - INFO - Loading data from csv file
2025-01-14 09:39:03,553 - INFO - Data loaded successfully


### Task 1 - Preprocessing

In [3]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [4]:
# Check for missing values
merged_data.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### 2.1 Preprocessing

In [5]:
# Convert the Date column to datetime format for test and train data
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_train['Date'] = pd.to_datetime(df_train['Date'])

# Extract holidays from train dataset
state_holidays = df_train[df_train['StateHoliday'] != '0']['Date'].unique()
state_holidays = np.array(sorted(state_holidays))  # Convert to a sorted numpy array for efficiency

# Helper functions
def get_weekday_or_weekend(date):
    return "Weekend" if date.weekday() >= 5 else "Weekday"

def days_to_next_holiday(date, holidays):
    future_holidays = holidays[holidays > date]
    return (future_holidays[0] - date).days if future_holidays.size > 0 else None

def days_after_last_holiday(date, holidays):
    past_holidays = holidays[holidays < date]
    return (date - past_holidays[-1]).days if past_holidays.size > 0 else None

def get_month_section(date):
    if date.day <= 10:
        return "Beginning"
    elif date.day <= 20:
        return "Mid"
    else:
        return "End"



# Add calculated columns to test and train data
for df in [df_test, df_train]:
    df['Weekday/Weekend'] = df['Date'].apply(get_weekday_or_weekend)
    df['Days to Next Holiday'] = df['Date'].apply(lambda x: days_to_next_holiday(x, state_holidays))
    df['Days After Last Holiday'] = df['Date'].apply(lambda x: days_after_last_holiday(x, state_holidays))
    df['Month Section'] = df['Date'].apply(get_month_section)



### 2.1.1 Extra features 

In [6]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"  
for df in [df_test, df_train]:
  df['Day of Year'] = df['Date'].dt.dayofyear
  df['Week Number'] = df['Date'].dt.isocalendar().week
  df['Is Quarter Start'] = df['Date'].dt.is_quarter_start
  df['Is Quarter End'] = df['Date'].dt.is_quarter_end
  df['Is Month Start'] = df['Date'].dt.is_month_start
  df['Is Month End'] = df['Date'].dt.is_month_end
  df['Season'] = df['Date'].apply(get_season)

  # Interaction and additional features
  df['Promo x Weekend'] = (df['Weekday/Weekend'] == 'Weekend') & (df['Promo'] == 1)
  df['Holiday x Season'] = df['Season'] + " - " + df['StateHoliday'].astype(str)

# Save the updated datasets
df_test.to_csv('test_updated.csv', index=False)
df_train.to_csv('train_updated.csv', index=False)
df_train.head()
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Weekday/Weekend,Days to Next Holiday,...,Month Section,Day of Year,Week Number,Is Quarter Start,Is Quarter End,Is Month Start,Is Month End,Season,Promo x Weekend,Holiday x Season
0,1,1,4,2015-09-17,1.0,1,0,0,Weekday,None,...,Mid,260,38,False,False,False,False,Autumn,False,Autumn - 0
1,2,3,4,2015-09-17,1.0,1,0,0,Weekday,None,...,Mid,260,38,False,False,False,False,Autumn,False,Autumn - 0
2,3,7,4,2015-09-17,1.0,1,0,0,Weekday,None,...,Mid,260,38,False,False,False,False,Autumn,False,Autumn - 0
3,4,8,4,2015-09-17,1.0,1,0,0,Weekday,None,...,Mid,260,38,False,False,False,False,Autumn,False,Autumn - 0
4,5,9,4,2015-09-17,1.0,1,0,0,Weekday,None,...,Mid,260,38,False,False,False,False,Autumn,False,Autumn - 0


### 2.1.2 Scaling the Data

In [7]:
from sklearn.preprocessing import StandardScaler


# Select numerical columns to scale (excluding non-numeric ones like 'Date' or categorical variables)
numerical_columns_train = df_train.select_dtypes(include=['int64', 'float64']).columns
numerical_columns_test = df_test.select_dtypes(include=['int64', 'float64']).columns


# Initialize the StandardScaler
scaler = StandardScaler()
#scaler_t = StandardScaler()

# Fit the scaler on training data and transform both train and test data
df_train[numerical_columns_train] = scaler.fit_transform(df_train[numerical_columns_train])
df_test[numerical_columns_test] = scaler.fit_transform(df_test[numerical_columns_test])

# Save the scaled datasets
df_train.to_csv('train_scaled.csv', index=False)
df_test.to_csv('test_scaled.csv', index=False)

print("Scaling complete! Scaled datasets saved as 'train_scaled.csv' and 'test_scaled.csv'.")

Scaling complete! Scaled datasets saved as 'train_scaled.csv' and 'test_scaled.csv'.


### 2.2 Building the model

In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Load the data
# Assuming you have already read `train_data`, `test_data`, and `store_data` into Pandas DataFrames
df_train = pd.read_csv('train_scaled.csv')

# Merge train_data and store_data
train_data = df_train.merge(df_store, on='Store', how='left')

# Sample a smaller subset of the data to speed up processing
#train_data = train_data.sample(frac=0.9, random_state=42)  # Use 10% of the data

# Feature engineering: Convert date column to datetime and extract useful features
train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data['Year'] = train_data['Date'].dt.year
train_data['Month'] = train_data['Date'].dt.month
train_data['Day'] = train_data['Date'].dt.day

# Ensure consistent data types
train_data['StateHoliday'] = train_data['StateHoliday'].astype(str)
train_data['StoreType'] = train_data['StoreType'].astype(str)
train_data['Assortment'] = train_data['Assortment'].astype(str)
train_data['PromoInterval'] = train_data['PromoInterval'].astype(str)

# Convert numeric columns with potential mixed types
numerical_cols = ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                  'Promo2SinceWeek', 'Promo2SinceYear']

for col in numerical_cols:
    train_data[col] = pd.to_numeric(train_data[col], errors='coerce')

# Select features and target
features = [
    'Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday',
    'Weekday/Weekend', 'Days to Next Holiday', 'Days After Last Holiday',
    'Month Section', 'Day of Year', 'Week Number', 'Is Quarter Start',
    'Is Quarter End', 'Is Month Start', 'Is Month End', 'Season',
    'Promo x Weekend', 'Holiday x Season', 'StoreType', 'Assortment',
    'CompetitionDistance', 'CompetitionOpenSinceMonth',
    'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
    'Promo2SinceYear', 'PromoInterval'
]

target = 'Sales'

# Split the data into train and test sets
X = train_data[features]
y = train_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing for numerical and categorical columns
numerical_features = [
    'Store', 'Promo', 'SchoolHoliday', 'Days to Next Holiday',
    'Days After Last Holiday', 'Day of Year', 'Week Number',
    'CompetitionDistance', 'CompetitionOpenSinceMonth',
    'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
    'Promo2SinceYear'
]

categorical_features = [
    'DayOfWeek', 'StateHoliday', 'Weekday/Weekend', 'Month Section',
    'Is Quarter Start', 'Is Quarter End', 'Is Month Start', 'Is Month End',
    'Season', 'Promo x Weekend', 'Holiday x Season', 'StoreType',
    'Assortment', 'PromoInterval'
]

# Numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Handle NaN after coercion
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Create the pipeline with a Random Forest Regressor
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Fit the model
model_pipeline.fit(X_train, y_train)

# Evaluate the model
predictions = model_pipeline.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 0.40779878015432297


### 2.3 Loss function
I chose Mean Absolute Error (MAE). This is a commonly used loss function for regression, which computes the absolute difference between predicted and actual values. It's easier to interpret and is less sensitive to outliers compared to Mean Squared Error (MSE).

In [18]:
from sklearn.metrics import mean_absolute_error

# Custom Loss Function: Mean Absolute Error (MAE)
def custom_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

### 2.4 Post Prediction Analysis: Exploring Feature Importance

In [33]:
# Feature Importance
feature_importance = model_pipeline.named_steps['regressor'].feature_importances_

# Estimating Confidence Interval using Bootstrapping
def bootstrap_predictions(model_pipeline, X_train, y_train, X_test, n_iterations=10):
    """
    Generate bootstrap predictions and calculate confidence intervals.
    """
    bootstrap_preds = []
    
    for _ in range(n_iterations):
        # Bootstrap sampling: Randomly sample with replacement
        X_sample, y_sample = resample(X_train, y_train)
        
        # Fit the model on the bootstrap sample
        model_pipeline.fit(X_sample, y_sample)
        
        # Predict on the test set
        y_pred = model_pipeline.predict(X_test)
        
        # Store the predictions
        bootstrap_preds.append(y_pred)
    
    # Convert the list of predictions into a numpy array for easier analysis
    bootstrap_preds = np.array(bootstrap_preds)
    
    # Calculate the confidence intervals (e.g., 95% CI)
    lower_bound = np.percentile(bootstrap_preds, 2.5, axis=0)
    upper_bound = np.percentile(bootstrap_preds, 97.5, axis=0)
    
    return lower_bound, upper_bound, bootstrap_preds

# Perform bootstrap and estimate confidence intervals
from sklearn.utils import resample

lower_bound, upper_bound, bootstrap_preds = bootstrap_predictions(model_pipeline, X_train, y_train, X_test)

# Ensure that the predictions and confidence intervals have the same shape
assert len(y_test) == bootstrap_preds.shape[1], "Mismatch in number of test samples between y_test and bootstrap_preds"

# Visualize the predictions and confidence intervals
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label="True Values", color="black", alpha=0.5)
plt.plot(bootstrap_preds.mean(axis=0), label="Predictions (mean)", color="blue")
plt.fill_between(np.arange(len(y_test)), lower_bound, upper_bound, color='gray', alpha=0.3, label="95% Confidence Interval")
plt.xlabel("Test Sample Index")
plt.ylabel("Predicted Sales")
plt.title("Predictions with 95% Confidence Interval")
plt.legend()
plt.show()



KeyboardInterrupt: 

### 2.5 Serealize models

In [29]:
import joblib
import datetime

# Assuming you already have your model trained, like 'model'

# 1. Get the current timestamp in the format "dd-mm-yyyy-HH-MM-SS-ff"
timestamp = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S-%f")

# 2. Define the model filename using the timestamp
model_filename = f"{timestamp}.pkl"

# 3. Save the model using joblib
joblib.dump(model_pipeline, model_filename)

print(f"Model saved as {model_filename}")

Model saved as 14-01-2025-14-47-55-548244.pkl


### 2.6 Building model with deep learning

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Assuming you have data in `data`, for example, stock prices, etc.
# Let's generate some dummy time series data for demonstration purposes
# In practice, replace this with your own dataset
data = np.sin(np.linspace(0, 100, 200))  # Just an example: a sine wave dataset

# Reshape the data for time series forecasting
data = data.reshape((len(data), 1))

# Use a TimeSeriesGenerator to create sequences of data for training
sequence_length = 10  # Number of time steps in each sequence
batch_size = 16

generator = TimeseriesGenerator(data, data, length=sequence_length, batch_size=batch_size)

# Define the LSTM model with two layers
model = Sequential()

# First LSTM layer with 50 units
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1), return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout for regularization

# Second LSTM layer with 50 units
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))  # Adding dropout for regularization

# Output layer
model.add(Dense(1))  # Predicting one value (can be modified for multi-output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(generator, epochs=10, verbose=1)

# After training, you can make predictions using the model
# Here we use the last part of the input data (in this case, the last 10 values)
test_input = data[-sequence_length:].reshape((1, sequence_length, 1))

# Predict the next value (next time step)
prediction = model.predict(test_input)
print(f'Prediction for the next value: {prediction[0][0]}')

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4912
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4395
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2733
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0826
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0329
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0268
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0272
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0226
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0175
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0195
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
Prediction for the next value: -0.023620009422302246


In [ ]:
# Feature Importance
feature_importance = model_pipeline.named_steps['regressor'].feature_importances_

# Estimating Confidence Interval using Bootstrapping
def bootstrap_predictions(model_pipeline, X_train, y_train, X_test, n_iterations=10):
    """
    Generate bootstrap predictions and calculate confidence intervals.
    """
    bootstrap_preds = []
    
    for _ in range(n_iterations):
        # Bootstrap sampling: Randomly sample with replacement
        X_sample, y_sample = resample(X_train, y_train)
        
        # Fit the model on the bootstrap sample
        model_pipeline.fit(X_sample, y_sample)
        
        # Predict on the test set
        y_pred = model_pipeline.predict(X_test)
        
        # Store the predictions
        bootstrap_preds.append(y_pred)
    
    # Convert the list of predictions into a numpy array for easier analysis
    bootstrap_preds = np.array(bootstrap_preds)
    
    # Calculate the confidence intervals (e.g., 95% CI)
    lower_bound = np.percentile(bootstrap_preds, 2.5, axis=0)
    upper_bound = np.percentile(bootstrap_preds, 97.5, axis=0)
    
    return lower_bound, upper_bound, bootstrap_preds

# Perform bootstrap and estimate confidence intervals
from sklearn.utils import resample

lower_bound, upper_bound, bootstrap_preds = bootstrap_predictions(model_pipeline, X_train, y_train, X_test)

# Ensure that the predictions and confidence intervals have the same shape
assert len(y_test) == bootstrap_preds.shape[1], "Mismatch in number of test samples between y_test and bootstrap_preds"

# Visualize the predictions and confidence intervals
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label="True Values", color="black", alpha=0.5)
plt.plot(bootstrap_preds.mean(axis=0), label="Predictions (mean)", color="blue")
plt.fill_between(np.arange(len(y_test)), lower_bound, upper_bound, color='gray', alpha=0.3, label="95% Confidence Interval")
plt.xlabel("Test Sample Index")
plt.ylabel("Predicted Sales")
plt.title("Predictions with 95% Confidence Interval")
plt.legend()
plt.show()



# Task 3 - Model Serving API Call